In [16]:
# Main packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import openpyxl

# Scikit-Learn
import sklearn as sk
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

# Tensorflow and Keras
import tensorflow as tf
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Flatten, Dropout
from tensorflow import keras

In [46]:
import tabulate
import xlrd
import math

# PREPROCESSING

## Preprocessing for input time series

In [9]:
COUNTRY_MAPPINGS = {"Austria": "AT_temperature"
                    ,"Bulgaria": "BG_temperature","France": "FR_temperature","Germany": "DE_temperature","Greece": "GR_temperature"
                    ,"Hungary": "HU_temperature","Italy": "IT_temperature","Portugal": "PT_temperature","Romania": "RO_temperature","Spain": "ES_temperature"
                    ,"Switzerland": "CH_temperature"}

MONTH_TO_DAY_MAPPINGS = {'1.0':31,'2.0':28,'3.0':31,'4.0':30,'5.0':31,'6.0':30,'7.0':31,'8.0':31,'9.0':30,'10.0':31,'11.0':30,'12.O':31}

def get_avg_temp(year, month, day, duration, country): # ('2003','08','15',4,'Austria')
    """
    Given a period and country in which a heatwave occurs, return the avg temperature of this period.
    """
    begin_date = str(year) + '-' + str(month) + '-' + str(day) + 'T00:00:00Z'
    print(begin_date)
    data = pd.read_csv('weather_data.csv')
    country = COUNTRY_MAPPINGS[country]
    df = data[['utc_timestamp',country]]
    begin_index = df.index[df['utc_timestamp'] == begin_date].tolist()[0]
    end_index = begin_index + (duration * 24)
    heat_wave_temps = df.iloc[begin_index:end_index]
    avg_temp = round(heat_wave_temps[country].sum()/len(heat_wave_temps.index),2)
    
    return avg_temp

def get_month_to_day(months):
    """
    Returns amount of days in a certain month.
    """
    return [MONTH_TO_DAY_MAPPINGS[str(m)] for m in months]

def plot(df_out,column_name):
    """
    Plot a certain column of a certain df.
    """
    x = df_out['Year'].to_numpy()
    y = df_out[column_name].to_numpy()
    plt.plot(x, y)
    plt.show()

## Get input time series and store them in pickles

In [22]:
def get_input_time_series(country):
    """
    Returns the 3 time series of a certain country between 1980 and 2019. Save as pickle file to later use with other data.
    -----------------------------------------------------------------
    Time Series 1: Amount heatwaves (per year)
    Time Series 2: Average temperature during all heatwaves (per year)
    Time Series 3: Average temperature in general
    """
    print(country)
    # Read in the full dataset, and only keep the heatwave data of the given country.
    colslist = ['Year','Disaster Subtype','Country','Start Month','Start Day','End Month','End Day']
    df = pd.read_excel('disaster_data.xlsx',usecols=colslist)
    df = df[df['Disaster Subtype'] == 'Heat wave']
    df = df[df['Country'] == country]
    df = df[df['Year'] >= 1980] # we only have temperature data from 1980-2019!
    df = df[df['Year'] <= 2019]
    year_list = np.linspace(1980,2019,2020-1980)
    df_out = pd.DataFrame({"Year": year_list})
    df_merged = pd.merge(df_out, df, how='left', on='Year')

    #-------------- Time Series 1: Amount heatwaves (per year) --------------#
    amount_list = [len(df[df['Year'] == y].index) for y in year_list]
    df_out['amount_heatwaves'] = amount_list

    #-------------- Time Series 2: Average temperature during all heatwaves (per year) --------------#

    # only begin- and end month and day given. So need to compute the exact duration of the heatwave which depends on the starting month.
    # Further, sometimes there's no start or end day given. If that's the case, take the avg temp of the month given.

    df = df.assign(month_difference=lambda row: row['End Month'] - row['Start Month'])
    df = df.assign(day_difference=lambda row: row['End Day'] - row['Start Day'])
    df['duration'] = np.where(df['day_difference'] > 0, df['day_difference'], 0)
    df['duration'] = np.where(df['day_difference'] == 0, 1, df['duration'])
    df['duration'] = np.where(df['day_difference'].isna(),get_month_to_day(df['Start Month']),df['duration']) # this can be still adjusted!
    df['duration'] = np.where(df['month_difference'] == 1, get_month_to_day(df['Start Month'])-df['Start Day'] + df['End Day'], df['duration'])
    df['duration'] = np.where(df['duration'].isna(), 30, df['duration'])
    df['duration'] = np.where(df['month_difference'] > 1, 30,df['duration']) # this can be still adjusted!
    df['start_day_for_duration'] = np.where(~df['Start Day'].isna(),df['Start Day'],1) # this can be still adjusted!

    # convert pd data to lists so we can perform the get_avg_temp function
    day_mapping = {1.0 :'01', 2.0 : '02', 3.0 : '03', 4.0 : '04', 5.0 : '05', 6.0 : '06', 7.0 : '07', 8.0 : '08', 9.0 : '09', 10.0 : '10', 11.0 : '11', 12.0 : '12', 13.0 : '13', 14.0 : '14', 15.0 : '15', 16.0 : '16', 17.0 : '17', 18.0 : '18', 19.0 : '19', 20.0 : '20', 21.0 : '21', 22.0 : '22', 23.0 : '23', 24.0 : '24', 25.0 : '25', 26.0 : '26', 27.0 : '27', 28.0 : '28', 29.0 : '29', 30.0 : '30', 31.0 : '31'}
    years_heatwave_list = [y for y in df['Year']]
    begin_month_list = [day_mapping[m] for m in df['Start Month']]
    begin_day_list = [day_mapping[day] for day in df['start_day_for_duration']]
    duration_list = [round(dur) for dur in df['duration']]
    avg_temps = [get_avg_temp(years_heatwave_list[i], begin_month_list[i], begin_day_list[i], duration_list[i], country) for i in range(len(years_heatwave_list))]

    # Compute avg temp --> list can be: [2003, 2006, 2015, 2018, 2019, 2019, 2019], take avg of the 2019 temps
    years_heatwave_array = np.array(years_heatwave_list)
    years_heatwave_array_unique = np.unique(years_heatwave_array)
    indices_array = [np.where(years_heatwave_array == year)[0] for year in years_heatwave_array_unique]
    df_out['avg_hw_temp'] = 0 # When there's no heatwave in a year, we put the avg temp = 0 for the time series
    
    for counter,indices in enumerate(indices_array):
        print(counter)
        temps = [avg_temps[ind] for ind in indices]
        avg_temp = round(sum(temps)/len(temps),2)
        print(avg_temp)
        df_out['avg_hw_temp'] = np.where(df_out['Year'] == years_heatwave_array_unique[counter],avg_temp,df_out['avg_hw_temp'])
    print(df_out.to_markdown())
    
    #-------------- Time Series 3: Average temperature in general --------------#
    avg_temps = [get_avg_temp(int(y), '01', '01', 365, country) for y in year_list]
    print(avg_temps)
    df_out['avg_temp'] = avg_temps
    print(df_out.to_markdown())
    #plot(df_out,'avg_temp')
    df_out.to_pickle(f"{country}.pkl")

    return df_out

#------------- Uncomment if you want to make the pickle files! (This is already done)---------------#
for i in COUNTRY_MAPPINGS:
    get_input_time_series(i)

Austria
2003-07-01T00:00:00Z
2007-07-01T00:00:00Z
2019-07-24T00:00:00Z
0
19.23
1
18.8
2
25.09
|    |   Year |   amount_heatwaves |   avg_hw_temp |
|---:|-------:|-------------------:|--------------:|
|  0 |   1980 |                  0 |          0    |
|  1 |   1981 |                  0 |          0    |
|  2 |   1982 |                  0 |          0    |
|  3 |   1983 |                  0 |          0    |
|  4 |   1984 |                  0 |          0    |
|  5 |   1985 |                  0 |          0    |
|  6 |   1986 |                  0 |          0    |
|  7 |   1987 |                  0 |          0    |
|  8 |   1988 |                  0 |          0    |
|  9 |   1989 |                  0 |          0    |
| 10 |   1990 |                  0 |          0    |
| 11 |   1991 |                  0 |          0    |
| 12 |   1992 |                  0 |          0    |
| 13 |   1993 |                  0 |          0    |
| 14 |   1994 |                  0 |          0    |
| 15 

1981-01-01T00:00:00Z
1982-01-01T00:00:00Z
1983-01-01T00:00:00Z
1984-01-01T00:00:00Z
1985-01-01T00:00:00Z
1986-01-01T00:00:00Z
1987-01-01T00:00:00Z
1988-01-01T00:00:00Z
1989-01-01T00:00:00Z
1990-01-01T00:00:00Z
1991-01-01T00:00:00Z
1992-01-01T00:00:00Z
1993-01-01T00:00:00Z
1994-01-01T00:00:00Z
1995-01-01T00:00:00Z
1996-01-01T00:00:00Z
1997-01-01T00:00:00Z
1998-01-01T00:00:00Z
1999-01-01T00:00:00Z
2000-01-01T00:00:00Z
2001-01-01T00:00:00Z
2002-01-01T00:00:00Z
2003-01-01T00:00:00Z
2004-01-01T00:00:00Z
2005-01-01T00:00:00Z
2006-01-01T00:00:00Z
2007-01-01T00:00:00Z
2008-01-01T00:00:00Z
2009-01-01T00:00:00Z
2010-01-01T00:00:00Z
2011-01-01T00:00:00Z
2012-01-01T00:00:00Z
2013-01-01T00:00:00Z
2014-01-01T00:00:00Z
2015-01-01T00:00:00Z
2016-01-01T00:00:00Z
2017-01-01T00:00:00Z
2018-01-01T00:00:00Z
2019-01-01T00:00:00Z
[9.46, 10.23, 10.11, 10.13, 10.0, 10.03, 10.4, 10.03, 10.61, 10.84, 11.4, 9.9, 10.59, 10.56, 11.9, 10.17, 9.98, 9.75, 10.79, 11.48, 11.46, 11.17, 11.0, 10.36, 10.9, 10.23, 10.82, 12

2003-08-01T00:00:00Z
2006-07-15T00:00:00Z
2018-07-01T00:00:00Z
2019-06-24T00:00:00Z
2019-07-24T00:00:00Z
0
21.26
1
23.09
2
21.0
3
25.06
|    |   Year |   amount_heatwaves |   avg_hw_temp |
|---:|-------:|-------------------:|--------------:|
|  0 |   1980 |                  0 |          0    |
|  1 |   1981 |                  0 |          0    |
|  2 |   1982 |                  0 |          0    |
|  3 |   1983 |                  0 |          0    |
|  4 |   1984 |                  0 |          0    |
|  5 |   1985 |                  0 |          0    |
|  6 |   1986 |                  0 |          0    |
|  7 |   1987 |                  0 |          0    |
|  8 |   1988 |                  0 |          0    |
|  9 |   1989 |                  0 |          0    |
| 10 |   1990 |                  0 |          0    |
| 11 |   1991 |                  0 |          0    |
| 12 |   1992 |                  0 |          0    |
| 13 |   1993 |                  0 |          0    |
| 14 |   1994 | 

1981-01-01T00:00:00Z
1982-01-01T00:00:00Z
1983-01-01T00:00:00Z
1984-01-01T00:00:00Z
1985-01-01T00:00:00Z
1986-01-01T00:00:00Z
1987-01-01T00:00:00Z
1988-01-01T00:00:00Z
1989-01-01T00:00:00Z
1990-01-01T00:00:00Z
1991-01-01T00:00:00Z
1992-01-01T00:00:00Z
1993-01-01T00:00:00Z
1994-01-01T00:00:00Z
1995-01-01T00:00:00Z
1996-01-01T00:00:00Z
1997-01-01T00:00:00Z
1998-01-01T00:00:00Z
1999-01-01T00:00:00Z
2000-01-01T00:00:00Z
2001-01-01T00:00:00Z
2002-01-01T00:00:00Z
2003-01-01T00:00:00Z
2004-01-01T00:00:00Z
2005-01-01T00:00:00Z
2006-01-01T00:00:00Z
2007-01-01T00:00:00Z
2008-01-01T00:00:00Z
2009-01-01T00:00:00Z
2010-01-01T00:00:00Z
2011-01-01T00:00:00Z
2012-01-01T00:00:00Z
2013-01-01T00:00:00Z
2014-01-01T00:00:00Z
2015-01-01T00:00:00Z
2016-01-01T00:00:00Z
2017-01-01T00:00:00Z
2018-01-01T00:00:00Z
2019-01-01T00:00:00Z
[15.26, 15.83, 15.33, 15.26, 15.54, 15.93, 15.74, 15.46, 15.99, 15.75, 16.45, 15.14, 15.55, 15.88, 16.71, 15.75, 15.38, 15.48, 16.14, 16.78, 16.4, 16.54, 16.31, 16.04, 16.13, 15.95,

2003-07-16T00:00:00Z
2007-06-01T00:00:00Z
2011-08-17T00:00:00Z
2018-08-01T00:00:00Z
2019-06-26T00:00:00Z
0
23.21
1
27.33
2
21.41
3
27.35
4
24.83
5
27.9
|    |   Year |   amount_heatwaves |   avg_hw_temp |
|---:|-------:|-------------------:|--------------:|
|  0 |   1980 |                  0 |          0    |
|  1 |   1981 |                  0 |          0    |
|  2 |   1982 |                  0 |          0    |
|  3 |   1983 |                  0 |          0    |
|  4 |   1984 |                  0 |          0    |
|  5 |   1985 |                  0 |          0    |
|  6 |   1986 |                  0 |          0    |
|  7 |   1987 |                  0 |          0    |
|  8 |   1988 |                  0 |          0    |
|  9 |   1989 |                  0 |          0    |
| 10 |   1990 |                  0 |          0    |
| 11 |   1991 |                  0 |          0    |
| 12 |   1992 |                  0 |          0    |
| 13 |   1993 |                  0 |          0    |


1981-01-01T00:00:00Z
1982-01-01T00:00:00Z
1983-01-01T00:00:00Z
1984-01-01T00:00:00Z
1985-01-01T00:00:00Z
1986-01-01T00:00:00Z
1987-01-01T00:00:00Z
1988-01-01T00:00:00Z
1989-01-01T00:00:00Z
1990-01-01T00:00:00Z
1991-01-01T00:00:00Z
1992-01-01T00:00:00Z
1993-01-01T00:00:00Z
1994-01-01T00:00:00Z
1995-01-01T00:00:00Z
1996-01-01T00:00:00Z
1997-01-01T00:00:00Z
1998-01-01T00:00:00Z
1999-01-01T00:00:00Z
2000-01-01T00:00:00Z
2001-01-01T00:00:00Z
2002-01-01T00:00:00Z
2003-01-01T00:00:00Z
2004-01-01T00:00:00Z
2005-01-01T00:00:00Z
2006-01-01T00:00:00Z
2007-01-01T00:00:00Z
2008-01-01T00:00:00Z
2009-01-01T00:00:00Z
2010-01-01T00:00:00Z
2011-01-01T00:00:00Z
2012-01-01T00:00:00Z
2013-01-01T00:00:00Z
2014-01-01T00:00:00Z
2015-01-01T00:00:00Z
2016-01-01T00:00:00Z
2017-01-01T00:00:00Z
2018-01-01T00:00:00Z
2019-01-01T00:00:00Z
[15.12, 15.73, 15.15, 15.18, 14.95, 15.33, 14.95, 15.81, 15.1, 16.36, 15.82, 15.23, 15.07, 14.71, 15.3, 16.49, 15.19, 16.24, 15.58, 15.21, 15.14, 15.1, 15.3, 15.39, 15.22, 15.34, 15

2003-08-01T00:00:00Z
2004-07-26T00:00:00Z
2006-07-15T00:00:00Z
2018-08-01T00:00:00Z
2019-06-26T00:00:00Z
0
25.82
1
27.93
2
24.75
3
26.21
4
25.01
5
25.94
|    |   Year |   amount_heatwaves |   avg_hw_temp |
|---:|-------:|-------------------:|--------------:|
|  0 |   1980 |                  0 |          0    |
|  1 |   1981 |                  0 |          0    |
|  2 |   1982 |                  0 |          0    |
|  3 |   1983 |                  0 |          0    |
|  4 |   1984 |                  0 |          0    |
|  5 |   1985 |                  0 |          0    |
|  6 |   1986 |                  0 |          0    |
|  7 |   1987 |                  0 |          0    |
|  8 |   1988 |                  0 |          0    |
|  9 |   1989 |                  0 |          0    |
| 10 |   1990 |                  0 |          0    |
| 11 |   1991 |                  0 |          0    |
| 12 |   1992 |                  0 |          0    |
| 13 |   1993 |                  0 |          0    |

1981-01-01T00:00:00Z
1982-01-01T00:00:00Z
1983-01-01T00:00:00Z
1984-01-01T00:00:00Z
1985-01-01T00:00:00Z
1986-01-01T00:00:00Z
1987-01-01T00:00:00Z
1988-01-01T00:00:00Z
1989-01-01T00:00:00Z
1990-01-01T00:00:00Z
1991-01-01T00:00:00Z
1992-01-01T00:00:00Z
1993-01-01T00:00:00Z
1994-01-01T00:00:00Z
1995-01-01T00:00:00Z
1996-01-01T00:00:00Z
1997-01-01T00:00:00Z
1998-01-01T00:00:00Z
1999-01-01T00:00:00Z
2000-01-01T00:00:00Z
2001-01-01T00:00:00Z
2002-01-01T00:00:00Z
2003-01-01T00:00:00Z
2004-01-01T00:00:00Z
2005-01-01T00:00:00Z
2006-01-01T00:00:00Z
2007-01-01T00:00:00Z
2008-01-01T00:00:00Z
2009-01-01T00:00:00Z
2010-01-01T00:00:00Z
2011-01-01T00:00:00Z
2012-01-01T00:00:00Z
2013-01-01T00:00:00Z
2014-01-01T00:00:00Z
2015-01-01T00:00:00Z
2016-01-01T00:00:00Z
2017-01-01T00:00:00Z
2018-01-01T00:00:00Z
2019-01-01T00:00:00Z
[6.08, 6.55, 7.53, 7.41, 6.44, 6.45, 6.96, 6.75, 7.52, 8.05, 7.77, 7.13, 7.45, 7.23, 8.39, 6.93, 6.25, 7.75, 7.25, 7.23, 7.94, 7.32, 8.01, 8.17, 7.28, 7.07, 7.88, 7.98, 7.46, 7.67, 

## Merging HeatWave data with production index data

In [23]:
def get_production_index_time_series(country, item):
    """
    Given a country and an item ['Agriculture' 'Cereals, Total' 'Food' 'Livestock' 'Milk, Total','Vegetables and Fruit Primary']
    return the time series of the product index of the item from 1980-2019.
    """
    df = pd.read_excel('World_prodvolumes.xls')
    all_countries = df['Area'].unique()
    all_items = df['Item'].unique()
    df = df[df['Area'] == country]
    df = df[df['Item'] == item]
    df = df[df['Year'] >= 1980]  # we only have temperature data from 1980-2019!
    df = df[df['Year'] <= 2019]
    df = df[['Year','Value']]
    df.fillna(df.mean(), inplace=True)

    print(all_countries)
    print(all_items)

    return df

def merge_time_series(country, item):
    """
    Read out a already computed pkl file of the 3 heat-related input time series and concatenate the product index time series to it.
    """
    df = pd.read_pickle(f"{country}.pkl")
    df_item = get_production_index_time_series(country, item)
    df_item = df_item.set_index(df.index) # indices have to be adjusted, otherwise NaN values!
    df['Value'] = df_item['Value']
    return df

# MODELING

## LSTM Model
### Importing the dataframe

In [32]:
evaluation_amount = 3 # amount of years we want to validate and test on.

#-------------- Import DataFrame --------------#
def import_data(country,product):
    df_total = merge_time_series(country,product)
    amount = len(df_total)
    df_train = df_total[:math.floor(amount -evaluation_amount)] # full training set
    df_test = df_total[math.floor(amount -evaluation_amount):] # test set, to do model evaluation at the end after hyperparameter tuning!
    return df_train,df_test,df_total

### Scale the data

In [33]:
#-------------- Scale the data: Needed for LSTM --------------#
def make_train_scaler(df_train):
    scaler = MinMaxScaler(feature_range=(0,1))
    scaled_train = scaler.fit_transform(df_train) #returns np.ndarray
    return scaled_train,scaler

### Split in training and test data

In [34]:
#-------------- Create X_train and y_train for the LSTM --------------#
def create_X_y(df,lag):
    df_length = df.shape[0]
    X = []
    y = []
    for i in range(lag,df_length): # eg if df = [1,2,3,4,5,6,7,...]
        X.append(df[i-lag:i]) # [1,2,3,4,5] (eg. lag value = 5)
        y.append(df[i][-1]) # [6] = next one!
    X = np.array(X)
    y = np.array(y)
    return X,y

### Make LSTM model

In [35]:
#-------------- Make the LSTM model --------------#
def make_model(X_train,y_train, amount_neurons,amount_dropout,lag):

    model = Sequential()

    # add Layer 1 with dropout: deactivate 10% of neurons at random during training to avoid overfitting
    model.add(LSTM(amount_neurons, activation='relu', input_shape=(lag, 4))) # we use 4 timeseries
    model.add(Dropout(amount_dropout))
    model.add(Dense(1))
    model.compile(optimizer='adam',loss='mse')

    #-------------- Fit the model on the training set --------------#
    model.fit(X_train,y_train,epochs=100)

    return model

### Make predictions

In [36]:
def predict(model, data_for_evaluation, train_scaler,lag): # data for evaluation means the data in the gap + val (or test)
    x_test_normal = data_for_evaluation.values
    scaled_test = train_scaler.transform(x_test_normal)  # scale test data with the scaler of the training data!
    X,y = create_X_y(scaled_test,lag) # y is not used anymore

    # predict the next values of the test set
    predictions_scaled = model.predict(X)
    # Add zeros to yr predictions, to fix the scaler error.
    zeros = np.zeros((evaluation_amount, 3))
    predictions_scaled = np.append(zeros, predictions_scaled, axis=1)

    # Re-transform the scaled data to the original format!
    predictions = train_scaler.inverse_transform(predictions_scaled)[:, [3]]
    predictions = predictions.flatten()
    y_true = np.array(data_for_evaluation['Value']) # get values of validation set
    print('true: ', y_true[-evaluation_amount:])
    print('-')
    print('pred: ',predictions)
    mse = mean_squared_error(y_true[-evaluation_amount:], predictions)
    print(f'mse = {mse}')

    return mse, predictions

### Forward validation

In [37]:
def forward_validation(amount_neurons, dropout, lag,df_train):
    amount_forward_splits = 4
    df_train_copy = df_train[:-(evaluation_amount+lag)]
    df_val_copy = df_train[-(evaluation_amount+lag):] # this is the data set we need to validate on, gap data included!
    mse_list = []
    for i in range(1,amount_forward_splits):
        print(f"fold {i}")
        scaled_train, train_scaler = make_train_scaler(df_train_copy)
        X_train, y_train = create_X_y(scaled_train,lag) # Train only on train fold, skip gap and val fold
        model = make_model(X_train, y_train, amount_neurons, dropout, lag)
        mse, _ = predict(model, df_val_copy, train_scaler,lag)
        mse_list.append(mse)
        df_train_copy = df_train_copy[:-evaluation_amount] # cut of validation amount for next training split
        df_val_copy = df_test[-(evaluation_amount+lag+(i*evaluation_amount)):-(i*evaluation_amount)]
    avg_mse = sum(mse_list) / len(mse_list)
    print('avg mse: ',avg_mse)

    return avg_mse

### Gridsearch for Hyperparameters

In [38]:
#-------------- Perform Grid Search to do hyperparameter tuning --------------#
def grid_search(df_train):
    grid_params = {'neurons':[10,20,40,50,100],'dropout_percentage':[0.05,0.1,0.2,0.5],'lags':[2,3,4,5]}
    best_model = {'mse':1000000,'neurons':0,'dropout_percentage':0,'lags':0} # here we will save our best model in
    for i in grid_params['neurons']:
        for j in grid_params['dropout_percentage']:
            for k in grid_params['lags']:
                print(f"neurons: {i}, drop: {j}, lags: {k}")
                avg_mse = forward_validation(i,j,k,df_train)
                if avg_mse < best_model['mse']:
                    print('better model found! :)')
                    best_model['mse'] = avg_mse
                    best_model['neurons'] = i
                    best_model['dropout_percentage'] = j
                    best_model['lags'] = k
    print('best model: ', best_model)

    return best_model

### Getting the results

In [45]:
#------------------------- Compute dictionary that contains all the predictions of the product indices of all relevant countries ---------------------------#
COUNTRY_LIST = ["Austria","Bulgaria","France","Germany","Greece","Hungary","Italy","Portugal","Romania","Spain","Switzerland"]
PRODUCT_LIST = ['Agriculture','Cereals, Total','Food', 'Livestock', 'Milk, Total','Vegetables and Fruit Primary']

product_values = [[0,0,0] for _ in range(len(PRODUCT_LIST))]
product_dict = dict(zip(PRODUCT_LIST, product_values))

# make output prediction dictionary
country_values = [product_dict for _ in range(len(COUNTRY_LIST))]
full_dict = dict(zip(COUNTRY_LIST, country_values)) #e.g. {"Albania":{"Agriculture":[v1,v2,v3],"Cereal":[v1,v2,v3],...}, "Austria":{"Agriculture":[v1,v2,v3],"Cereal":[v1,v2,v3],...}}
output_dict = {}

def compute_dictionary():
    for c in COUNTRY_LIST:
        product_values = [[0, 0, 0] for _ in range(len(PRODUCT_LIST))]
        product_dict = dict(zip(PRODUCT_LIST, product_values))
        country_dict = dict(zip([c], [product_dict]))
        for p in PRODUCT_LIST:
            # -------------- Use best model parameters to train on full data set, evaluate on independant test set --------------#
            df_train, df_test,df_total = import_data(c,p)
            best_model = grid_search(df_train)
            lags = best_model['lags']
            train = df_train[:-(evaluation_amount + lags)]
            test = df_train[-(evaluation_amount + lags):]  # this is the data set we need to test on, gap data included!
            scaled_train, train_scaler = make_train_scaler(train)
            X_train, y_train = create_X_y(scaled_train, best_model['lags'])
            m = make_model(X_train, y_train, best_model['neurons'], best_model['dropout_percentage'],best_model['lags'])
            mse, predictions = predict(m, test, train_scaler,lags)  # scaling of test data happens in this function
            print('best model predictions: ',predictions )
            print('mse on test set: ', mse)
            print(c,p)
            print(full_dict[c][p])
            country_dict[c][p] = predictions
        output_dict[c] = country_dict
    print(output_dict)
#------------ Uncomment to do hyperparameter tuning!------------- #
compute_dictionary()

['Afghanistan' 'Albania' 'Algeria' 'Angola' 'Antigua and Barbuda'
 'Argentina' 'Armenia' 'Australia' 'Austria' 'Azerbaijan' 'Bahamas'
 'Bahrain' 'Bangladesh' 'Barbados' 'Belarus' 'Belgium'
 'Belgium-Luxembourg' 'Belize' 'Benin' 'Bhutan'
 'Bolivia (Plurinational State of)' 'Bosnia and Herzegovina' 'Botswana'
 'Brazil' 'Brunei Darussalam' 'Bulgaria' 'Burkina Faso' 'Burundi'
 'Cabo Verde' 'Cambodia' 'Cameroon' 'Canada' 'Central African Republic'
 'Chad' 'Channel Islands' 'Chile' 'China, Hong Kong SAR'
 'China, Macao SAR' 'China, mainland' 'China, Taiwan Province of'
 'Colombia' 'Comoros' 'Congo' 'Cook Islands' 'Costa Rica' "C?te d'Ivoire"
 'Croatia' 'Cuba' 'Cyprus' 'Czechia' 'Czechoslovakia'
 "Democratic People's Republic of Korea"
 'Democratic Republic of the Congo' 'Denmark' 'Djibouti' 'Dominica'
 'Dominican Republic' 'Ecuador' 'Egypt' 'El Salvador' 'Equatorial Guinea'
 'Eritrea' 'Estonia' 'Eswatini' 'Ethiopia' 'Ethiopia PDR' 'Faroe Islands'
 'Fiji' 'Finland' 'France' 'French Guyana' 'F

2022-07-25 18:07:34.525103: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-07-25 18:07:34.527241: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/100


2022-07-25 18:07:35.516080: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


ValueError: in user code:

    File "/Users/pjcnudde/tensorflowMDA/env/lib/python3.10/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/Users/pjcnudde/tensorflowMDA/env/lib/python3.10/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/pjcnudde/tensorflowMDA/env/lib/python3.10/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/Users/pjcnudde/tensorflowMDA/env/lib/python3.10/site-packages/keras/engine/training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "/Users/pjcnudde/tensorflowMDA/env/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/pjcnudde/tensorflowMDA/env/lib/python3.10/site-packages/k